In [101]:
from sympy import *

In [110]:
u, v = symbols('u, v')
r = symbols('r')

x = r * sin(u) * cos(v)
y = r * sin(u) * sin(v)
z = r * cos(u)

surface = Matrix([[x], [y], [z]])

# x_u = simplify(surface.diff(u))
# x_v = simplify(surface.diff(v))
# jacobian = simplify(Matrix.hstack(x_u, x_v))
# display(jacobian)
# simplify(jacobian.T * jacobian)
coordinates = [u, v]
jacobian = simplify(surface.jacobian(Matrix(coordinates)))
metric = simplify(jacobian.T * jacobian)

metricInv = simplify(metric.inv())

coorindate_count = 2
cristoffel = [ 
    [ 
        [Integer(0), Integer(0)],
        [Integer(0), Integer(0)] 
    ],
    [ 
        [Integer(0), Integer(0)],
        [Integer(0), Integer(0)] 
    ] 
]

# https://math.stackexchange.com/questions/1720424/how-to-best-calculate-christoffel-symbols-of-metric-du2-g2u-dv2
for i in range(coorindate_count):
    for j in range(coorindate_count):
        x_j = coordinates[j]
        for k in range(coorindate_count):
            x_k = coordinates[k]
            for m in range(coorindate_count):
                x_m = coordinates[m]
                cristoffel[i][j][k] += metricInv[i, m] * surface.diff(x_j, x_k).dot(surface.diff(x_m))

for i in range(coorindate_count):
    cristoffel[i] = simplify(Matrix(cristoffel[i]))

print('Cristoffel symbols')
display(cristoffel[0])
display(cristoffel[1])

x_u = jacobian.col(0)
x_v = jacobian.col(1)
print('x_u')
display(x_u)

print('x_v')
display(x_v)

# https://stackoverflow.com/questions/39691325/display-a-matrix-with-putting-a-common-factor-in-sympy
normal = simplify(x_u.cross(x_v))
normal = expand_trig(normal)
normal /= gcd(tuple(normal))
print('normal')
display(normal)

Cristoffel symbols


Matrix([
[0,           0],
[0, -sin(2*u)/2]])

Matrix([
[       0, 1/tan(u)],
[1/tan(u),        0]])

x_u


Matrix([
[r*cos(u)*cos(v)],
[r*sin(v)*cos(u)],
[      -r*sin(u)]])

x_v


Matrix([
[-r*sin(u)*sin(v)],
[ r*sin(u)*cos(v)],
[               0]])

normal


Matrix([
[sin(u)*cos(v)],
[sin(u)*sin(v)],
[       cos(u)]])